## Option Pricing Using Monte Carlo Simulation

Generate $n$ _iid_ Bernoulli $(p)$ random variables $B_1, ..., B_n$ and use as the estimate $$ \hat{p}= \frac{1}{n}\sum_{i=1}^{n}B_i $$ 

In [1]:
# This function generates a list of n trials of Bernoulli random variables with probability distribution p

import random


def bernoulli_list(p, n):
    rv_list = []
    
    while len(rv_list) < n:
        U = random.random()
        
        if U <=p:
            rv_list.append(1)
        else:
            rv_list.append(0)
            
    return rv_list

In [2]:
# Test the bernoulli_list function

print(bernoulli_list(0.5, 10))

[0, 0, 1, 1, 1, 0, 0, 0, 0, 1]


In [3]:
# This function estimates the probability (p_hat) of an event occuring from a list of Bernoulli random variables with
# probability distrubtion p and n trials

def bernoulli_est(p, n):
    
    rv_list = bernoulli_list(p, n)

    n = len(rv_list)

    rv_sum = sum(rv_list)

    p_hat = rv_sum / n

    return p_hat

In [4]:
# Caculate the probability estimate for a distribution with probability p=0.5 and observations n=100
print(bernoulli_est(0.5, 100))

0.52


In [5]:
# Caculate the probability estimate for a distribution with probability p=0.5 and observations n=1000
print(bernoulli_est(0.5, 1000))

0.509


In [6]:
# Caculate the probability estimate for a distribution with probability p=0.5 and observations n=10000
print(bernoulli_est(0.5, 10000))

0.4995


In [7]:
# Caculate the probability estimate for a distribution with probability p=0.75 and observations n=100
print(bernoulli_est(0.75, 100))

0.81


In [8]:
# Caculate the probability estimate for a distribution with probability p=0.75 and observations n=1000
print(bernoulli_est(0.75, 1000))

0.762


In [9]:
# Caculate the probability estimate for a distribution with probability p=0.75 and observations n=10000
print(bernoulli_est(0.75, 10000))

0.7466


Recall the formula for computing the price $C_0$ of an option (derivative of the BLM stock prices) That yields a payoff at time $T$, denote by $C_T$:
$$ C_0 = \frac{1}{(1+r)^T}E^*(C_T), $$  

where $*$ refers to the fact that we must use the value $p^*$ instead of the original (real) $P$ for the up/down probability of the BLM. (The real value of $P$ is not needed for priciing.) Also recall that for $C_T = (S_T - K)^+$, the European call option, the expected value, $E^*(S_T-K)^+$ can be computed explicitly yielding the famous Black-Scholes-Merton option pricing formula:  
$$ C_0 = \frac{1}{(1+r)^T}\sum_{k=0}^{T} \left( \begin{array}{c}
T \\ k  \end{array} \right )(p^*)^k(1-p^*)^{T-k}(u^k d^{T-k}S_0-K)^+ . $$  

$$ p^* = \frac{1+r-d}{u-d} . $$  

For the Monte Carlo, use $n=100, n=1000, n=10,000$ _iid_ copies of $C_T$ (for averaging) to see how it gets more accurate as $n$ increases.

In [10]:
import random

# Parameters
T = 10  # Time
r = 0.05  # Risk-free rate
u = 1.15
d = 1.01
p = 0.05
S_0 = 50  # Stock price
K = 70  # Strike price


# This function generates a Y random variable (u or d) used in the Binomial Lattice Model

def Y_rv(p, u, d):
    U = random.random()

    if U <= p:
        return u
    else:
        return d

In [11]:
# This function simulates one path of a Binomial Lattice Model up to time T
# BLMpath represents the day-by-day changes to a price of a stock
def blm_path(T, u, d, p, S_0):
    
    #Initialize an empty array for the stock value at each discrete time interval, blm_path
    blm_path = [0 for k in range(T + 1)]

    #Set the first entry to our initial stock value (S_0)
    blm_path[0] = S_0

    #Generate our path by sampling from Y one at a time
    for k in range(1, T + 1):
        blm_path[k] = blm_path[k - 1] * Y_rv(p, u, d)

    return blm_path

In [12]:
# This function uses a Monte Carlo simulation of Binomial Lattice Model paths, with n iid itterations, to calculate the 
# price of an option
def blm_mc(T, r, u, d, S_0, K, n):
    # Calculate p_star, p_star
    p_star = (1 + r - d) / (u - d)
    
    # Initialize the sum of the last node on the Binomial Lattice Model paths, sum_value
    sum_value = 0
    
    # Simulate BLM paths and calculate the option price n times
    for i in range(n):
        # The simulated stock value is the stock value at time T, stock_value
        stock_value = blm_path(T, u, d, p_star, S_0)[-1]
        
        # Calculate the option value by subtracting the stock value at T by the strike price, C_T
        C_T = stock_value - K
        
        # Sum the option prices for averaging
        sum_value += C_T
        
    # Divide sum_value by n to receive an average value of the option value, avg_value
    avg_value = sum_value / n
    
    # Calculate the option price at time 0 by dividing the average option value by the discount, C_0
    C_0 = (1 / (1 + r)**T) * avg_value
    
    return C_0

In [13]:
# Simulate the Binomial Lattice Model 100 times
print(blm_mc(T, r, u, d, S_0, K, 100))

5.759834690027413


In [14]:
# Simulate the Binomial Lattice Model 1000 times
print(blm_mc(T, r, u, d, S_0, K, 1000))

7.304002234725253


In [15]:
# Simulate the Binomial Lattice Model 10000 times
print(blm_mc(T, r, u, d, S_0, K, 10000))

7.0230138667757185


In [16]:
import math

# This formula

def black_scholes_merton(T, r, u, d, S_0, K):
    # Calculate p_star, p_star
    p_star = (1 + r - d) / (u - d)
    
    # Sum the value from 0 to T, sum
    sum = 0
    
    # Sum from 0 to T
    for k in range(T+1):
        sum += (math.factorial(T) / (math.factorial(k) * math.factorial(T - k))) * p_star**k * (1 - p_star)**(T - k) *\
                (u**k * d**(T - k) * S_0 - K)
        
    # Calculate the option price by dividing sum by the discount, C_0
    C_0 = (1 / (1 + r)**T) * sum
    
    return C_0

In [17]:
# Test the black_scholes_merton function
print(black_scholes_merton(T, r, u, d, S_0, K))

7.026072252146863


__Asian Call Option:__ The payoff $C_T$ at atime $T$ is based on the average value of the stock over the $T$ time units:
$$ C_T = \left( \frac{1}{T} \sum_{i=1}^{T} S_i-K \right)^+ $$  

Use $n=100, n=1000, n=10,000$ _iid_ copies of $C_T$ (for averaging).

In [18]:
# Parameters
T = 10  # Time
r = 0.05  # Risk-free rate
u = 1.15
d = 1.01
S_0 = 50  # Stock price
K = 70  # Strike price

# This function calculates the price of Asian call options using Binomial Lattice Model path simulation, and averages the
# stock price n times

def asian_call(T, r, u, d, S_0, K, n):
    # Calculate p_star
    p_star = (1 + r - d) / (u - d)
    
    # Initialize the sum of the totals of the Binomial Lattice Model paths, sum_value
    sum_total = 0

    # Simulate BLM paths n times, sum all of the nodes, and divide by T
    for i in range(n):
        
        # Sum the BLM simulated stock values at each time interval, blm_sum
        blm_sum = sum(blm_path(T, u, d, p_star, S_0))
        
        # Calculate the average Asian stock value over T by dividing the summed stock values by T, S_T
        S_T = blm_sum / T
    
        # Calculate the Asian call option price by subtracting the average stock value by the strike price, C_T
        C_T = S_T - K
    
        # sum the totals of the Asian stock values to find the average
        sum_total += C_T
        
    # Calculate the average option price by dividing the sum_total by n, avg_value
    avg_value = sum_total / n
        
    return avg_value

In [19]:
# Simulate the Asian call option pricing 100 times
print(asian_call(T, r, u, d, S_0, K, 100))

-1.0838878538979646


In [20]:
# Simulate the Asian call option pricing 1000 times
print(asian_call(T, r, u, d, S_0, K, 1000))

1.0623522243867591


In [21]:
# Simulate the Asian call option pricing 10000 times
print(asian_call(T, r, u, d, S_0, K, 10000))

1.0338101742929227


__Barrier Option:__ 

$$ C_T = (S_T -K)^+ I \{ S_{n_1} \geq b, S_{n_2} \geq b \}.$$ 

Use $n=100, n=1000, n=10,000$ _iid_ copies of $C_T$ (for averaging).

In the above, recall that for any event $A$, $I\{A\}$ denotes the _indicator_ random variable defined by  
$$ I\{A\} = \left\{ \begin{array}{ll} 1 & \mbox{if $A$ occurs,} \\ 0 & \mbox{if $A$ does not occur.} \end{array} \right. $$

  
Here, $A = \{S_{n_1} \geq b, S_{n_2} \geq b\}$.


In [22]:
# Parameters
T = 10  # Time
r = 0.05  # Risk-free rate
u = 1.15
d = 1.01
S_0 = 50  # Stock price
K = 70  # Strike price
n_1 = 4  # first barrier time
n_2 = 6  # second barrier time
b = 66  # barrier price

# This function calculates the price of barrier options, with parameters n_1, n_2, and b, using Binomial Lattice Model path 
# simulation, and averages the stock price n times

def barrier_option(T, r, u, d, S_0, K, n_1, n_2, b, n):
    # Calculate p_star
    p_star = (1 + r - d) / (u - d)

    # Initialize the sum of the last node on the Binomial Lattice Model paths, sum_value
    sum_value = 0

    # Simulate BLM paths n times and sum up the final value
    for i in range(n):
        # Create a list of the simulated BLM path
        blm_nodes = blm_path(T, u, d, p_star, S_0)

        S_T = blm_nodes[-1]

        option_price = S_T - K

        # Check that the stock values at times n_1 and n_2 are above the barrier price
        # If either of the stock values are below b, the option price is zero
        if blm_nodes[n_1] < b or blm_nodes[n_2] < b:
            option_price = 0

        sum_value += option_price

    # Divide sum_value by n to receive an average value of the option prices, avg_value       
    avg_value = sum_value / n

    return avg_value

In [23]:
# Simulate the barrier option pricing 100 times
print(barrier_option(T, r, u, d, S_0, K, n_1, n_2, b, 100))

7.415860505859539


In [24]:
# Simulate the barrier option pricing 1000 times
print(barrier_option(T, r, u, d, S_0, K, n_1, n_2, b, 1000))

8.184419090411623


In [25]:
# Simulate the barrier option pricing 10000 times
print(barrier_option(T, r, u, d, S_0, K, n_1, n_2, b, 10000))

7.613178638447524
